In [10]:
 %env
#Imports
import nltk
import os
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle

#Loading Data
with open("intents.json") as file:
	data = json.load(file)
  
#Initializing empty lists
words = []
labels = []
docs_x = []
docs_y = []

#Looping through our data
for intent in data['intents']:
	for pattern in intent['patterns']:
		pattern = pattern.lower()
    		#Creating a list of words
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent['tag'])

	if intent['tag'] not in labels:
	  labels.append(intent['tag'])
  
stemmer = LancasterStemmer()
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]
for x,doc in enumerate(docs_x):
	bag = []
	wrds = [stemmer.stem(w) for w in doc]
	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1
	training.append(bag)
	output.append(output_row)
#Converting training data into NumPy arrays
training = np.array(training)
output = np.array(output)

#Saving data to disk
with open("data.pickle","wb") as f:
	pickle.dump((words, labels, training, output),f)
 
tf.reset_default_graph()

net = tflearn.input_data(shape = [None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch = 200, batch_size = 8, show_metric = True)
model.save("model.tflearn")

ModuleNotFoundError: No module named 'tensorflow.contrib'

In [8]:
%env

{'TERM_PROGRAM': 'Apple_Terminal',
 'TERM': 'xterm-color',
 'SHELL': '/bin/bash',
 'TMPDIR': '/var/folders/hk/fh34_pyx5dlcj_hr_pj3vc3m0000gn/T/',
 'CONDA_SHLVL': '1',
 'TERM_PROGRAM_VERSION': '433',
 'CONDA_PROMPT_MODIFIER': '(base) ',
 'TERM_SESSION_ID': 'FAF4F183-F344-4972-86C5-DE0E87B34723',
 'USER': 'rishkum',
 'CONDA_EXE': '/opt/anaconda3/bin/conda',
 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.WUsfUPfMUp/Listeners',
 '_CE_CONDA': '',
 'PATH': '/opt/anaconda3/bin:/opt/anaconda3/condabin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/share/dotnet:/opt/X11/bin:~/.dotnet/tools:/Library/Frameworks/Mono.framework/Versions/Current/Commands',
 'CONDA_PREFIX': '/opt/anaconda3',
 'PWD': '/Users/rishkum',
 'LANG': 'en_GB.UTF-8',
 'XPC_FLAGS': '0x0',
 'XPC_SERVICE_NAME': '0',
 '_CE_M': '',
 'HOME': '/Users/rishkum',
 'SHLVL': '2',
 'LOGNAME': 'rishkum',
 'CONDA_PYTHON_EXE': '/opt/anaconda3/bin/python',
 'CONDA_DEFAULT_ENV': 'base',
 'DISPLAY': '/private/tmp/com.apple.launchd.IJ